In [2]:
!pip install -r requirements.txt

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [3]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 

In [4]:
raw_data= pd.read_csv("data//breast_cancer_dataset_dse.csv")
raw_data.head()

,CENTRO,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,NUT81,NUT82,NUT83,NUT84,NUT85,NUT86,NUT87,NUT88,NUT89,NUT90
0,1,10001,1,4,2,2,491,-9532,57,2,...,5648.296782,3211.984111,5293.483394,9693.117448,24592.425850,4435.469432,8336.245970,5491.056427,92.785579,71.944429
1,1,10001,2,12,2,4,492,-14138,70,2,...,2821.803323,1633.620478,2759.095141,5193.159895,10914.074588,2170.876027,4026.198830,2813.115847,45.196500,33.327571
2,1,10002,1,10,2,10,891,-10885,61,4,...,5182.036804,2768.285972,4673.497237,8435.682215,18817.064964,4027.165050,5520.428967,4191.977318,72.617601,57.553526
3,1,10002,2,12,2,2,592,-5872,48,2,...,7525.644453,4443.970135,7546.378739,13406.852180,30814.012101,5692.820114,12382.319142,7979.416594,118.245746,91.127254
4,1,10003,1,10,2,2,891,-7136,51,2,...,3333.196596,1766.302257,3108.772785,5701.329841,14685.569279,2684.650272,4775.958162,3194.035958,65.571295,50.351497


- v23 (dati non reliable, da considerare per eliminarne? boh)
- v24
- farm 40 
- v6 v7 --- altrimenti togleire v8 e usare v6 e v7 per calcolare età di v17 (età diagnosi)
- v9 a v15 vedere utilità se no togliere 
- v16 da valutare (main diagnosis)
- fum8- fum17 da valutare 
- fis1 - fis8 (lack data in b/ween)
- antr5 
- antr8 - antr11 e antr13 
- da cond1 a open8 -> set a parte > diete 
- alc resto da togliere
- nut50-nut52 e nut55-nut60 mancano i nomi delle variabili 


*da fare*
- v8-fum2
- fum9-fum8
- alc4+alc8+alc12+alc16+alc20+alc25
- medial history da valutare statistically 
- da fam 1 a 32 capire come gestirle (o raggrupparle, o creare 4 variabili vedi chatgpt o valutare come)


In [7]:
alcs_keep = {"ALC4", "ALC8", "ALC12", "ALC16", "ALC20", "ALC25"}
diets_col= ["COND1", "COND2", "COND3", "COND4", "COND5", "COND6", "COND7", "COND8", "COND9", "COND10", "COND11", "COND12",
    "COND13", "COND14", "COND15", "COND16", "COND17", "PASTI1", "PASTI2", "PASTI3", "PASTI4", "PASTI5",
    "ALI1", "ALI2", "ALI3", "ALI4", "ALI5", "ALI6", "ALI7", "ALI8", "ALI9", "ALI10", "ALI11", "CODOP1", "OPEN1",
    "ALI12", "ALI13", "ALI14", "ALI15", "ALI16", "PORZ17", "ALI17", "PORZ18", "ALI18", "PORZ19", "ALI19", "PORZ20",
    "ALI20", "PORZ21", "ALI21", "PORZ22", "ALI22", "PORZ23", "ALI23", "PORZ24", "ALI24", "CODOP2", "POROP2", "OPEN2",
    "ALI25", "PORIAS1", "RIAS1", "PORIAS2", "RIAS2", "ALI26", "ALI27", "PORZ28", "ALI28", "PORZ29", "ALI29", "PORZ30",
    "ALI30", "PORZ31", "ALI31", "PORZ32", "ALI32", "PORZ33", "ALI33", "PORZ34", "ALI34", "PORZ35", "ALI35", "PORZ36",
    "ALI36", "PORZ37", "ALI37", "PORZ38", "ALI38", "PORZ39", "ALI39", "PORZ40", "ALI40", "PORZ41", "ALI41", "PORIAS4",
    "RIAS4", "PORZ42", "ALI42", "PORZ43", "ALI43", "PORZ44", "ALI44", "CODOP3", "POROP3", "OPEN3", "PORIAS3", "RIAS3",
    "PORZ45", "ALI45", "PORZ46", "ALI46", "PORZ47", "ALI47", "PORZ48", "ALI48", "PORZ49", "ALI49", "PORZ50", "ALI50",
    "PORZ51", "ALI51", "ALI52", "MESI52", "PORZ53", "ALI53", "MESI53", "PORZ54", "ALI54", "MESI54", "PORZ55", "ALI55",
    "MESI55", "PORZ56", "ALI56", "MESI56", "PORZ57", "ALI57", "MESI57", "CODOP4", "POROP4", "OPEN4", "MESOP4",
    "CODOP5", "POROP5", "OPEN5", "MESOP5", "PORIAS5", "RIAS5", "PORIAS6", "RIAS6", "CODREG1", "POREG1", "REG1",
    "CODREG2", "POREG2", "REG2", "ALI58", "ALI59", "ALI60", "ALI61", "ALI62", "ALI63", "ALI64", "ALI65", "MESI65",
    "ALI66", "MESI66", "ALI67", "MESI67", "ALI68", "MESI68", "ALI69", "MESI69", "CODOP6", "OPEN6", "MESOP6",
    "CODOP7", "OPEN7", "MESOP7", "RIAS7", "ALI70", "ALI71", "ALI72", "PORZ73", "ALI73", "PORZ74", "ALI74",
    "ALI75", "ALI76", "ALI77", "ALI78", "MESI78", "CODOP8", "POROP8", "OPEN8"
]

cols_to_drop = [f"ALC{i}" for i in range(1, 28) if f"ALC{i}" not in alcs_keep] + [
    "V24", "FARM40", "FIS1", "FIS2", "FIS3", "FIS4", "FIS5", "FIS6", "FIS7", "FIS8",
    "ANTR5", "ANTR8", "ANTR9", "ANTR10", "ANTR11", "ANTR13", "NUT50", "NUT51","NUT51","NUT55",
    "NUT56","NUT57","NUT58","NUT59","NUT60",
] + diets_col

data = raw_data.drop(cols_to_drop, axis=1, errors="ignore")

PRESENTAZIONE: Ricordiamoci di spiegare come mai queste variabili sono state ignorate

In [8]:
data

,CENTRO,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,NUT81,NUT82,NUT83,NUT84,NUT85,NUT86,NUT87,NUT88,NUT89,NUT90
0,1,10001,1,4,2,2,491,-9532,57,2,...,5648.296782,3211.984111,5293.483394,9693.117448,24592.425850,4435.469432,8336.245970,5491.056427,92.785579,71.944429
1,1,10001,2,12,2,4,492,-14138,70,2,...,2821.803323,1633.620478,2759.095141,5193.159895,10914.074588,2170.876027,4026.198830,2813.115847,45.196500,33.327571
2,1,10002,1,10,2,10,891,-10885,61,4,...,5182.036804,2768.285972,4673.497237,8435.682215,18817.064964,4027.165050,5520.428967,4191.977318,72.617601,57.553526
3,1,10002,2,12,2,2,592,-5872,48,2,...,7525.644453,4443.970135,7546.378739,13406.852180,30814.012101,5692.820114,12382.319142,7979.416594,118.245746,91.127254
4,1,10003,1,10,2,2,891,-7136,51,2,...,3333.196596,1766.302257,3108.772785,5701.329841,14685.569279,2684.650272,4775.958162,3194.035958,65.571295,50.351497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5152,7,70086,2,14,2,12,294,-14640,74,4,...,4809.116436,2645.048996,4463.223475,8669.116463,21642.053156,3760.635672,6939.169466,4661.511688,65.959292,50.958445
5153,7,70087,2,14,2,12,294,-12771,69,2,...,6864.058656,3825.790086,6523.646366,11722.261381,35078.468430,5858.961632,11196.591061,6761.244768,112.616708,86.627207
5154,7,70088,2,14,2,12,294,-13424,70,4,...,2919.270469,1554.444277,2697.311293,5105.081750,10997.028813,2268.770773,3549.838482,2541.161477,47.411014,36.361197
5155,7,70089,2,14,2,4,294,-13679,71,4,...,1897.566155,1150.334504,1805.547359,3603.283269,8955.762793,1446.452641,3319.249391,2129.870453,36.938776,26.708642
